**Prune**

For input:

`Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.`

1. prune by upper bound of geodes collected (_76m 58.5s_)
   ```
   minute 1: 1 states, bestScore=0
   minute 2: 1 states, bestScore=0
   minute 3: 1 states, bestScore=0
   minute 4: 2 states, bestScore=0
   minute 5: 4 states, bestScore=0
   minute 6: 9 states, bestScore=0
   minute 7: 22 states, bestScore=0
   minute 8: 57 states, bestScore=0
   minute 9: 146 states, bestScore=0
   minute 10: 367 states, bestScore=0
   minute 11: 843 states, bestScore=0
   minute 12: 1839 states, bestScore=0
   minute 13: 4130 states, bestScore=0
   minute 14: 9676 states, bestScore=0
   minute 15: 23441 states, bestScore=0
   minute 16: 57115 states, bestScore=0
   minute 17: 136000 states, bestScore=0
   minute 18: 308332 states, bestScore=0
   minute 19: 663160 states, bestScore=6
   minute 20: 1394520 states, bestScore=6
   minute 21: 2956736 states, bestScore=10
   minute 22: 805507 states, bestScore=10
   minute 23: 129141 states, bestScore=12
   minute 24: 1340 states, bestScore=12
   ```
2. Do not construct more robots than needed.
   1. If we can only consume at most 3 ore per minute, we need at most 3 ore robot, this optimization prune lots of branches. (_3m 42.6s_)
   2. If currently collected ore and ore robots can provide enough resources - even when we build the type of robot that costs most ore in every remaining minute, we don't need to explore the states that have more ore robots. (_2m 23.7s_)
   ```
   minute 1: 1 states, bestScore=0
   minute 2: 1 states, bestScore=0
   minute 3: 1 states, bestScore=0
   minute 4: 2 states, bestScore=0
   minute 5: 4 states, bestScore=0
   minute 6: 9 states, bestScore=0
   minute 7: 20 states, bestScore=0
   minute 8: 46 states, bestScore=0
   minute 9: 101 states, bestScore=0
   minute 10: 210 states, bestScore=0
   minute 11: 396 states, bestScore=0
   minute 12: 728 states, bestScore=0
   minute 13: 1416 states, bestScore=0
   minute 14: 2919 states, bestScore=0
   minute 15: 6196 states, bestScore=0
   minute 16: 13165 states, bestScore=0
   minute 17: 27166 states, bestScore=0
   minute 18: 53080 states, bestScore=0
   minute 19: 98078 states, bestScore=6
   minute 20: 177920 states, bestScore=6
   minute 21: 327335 states, bestScore=10
   minute 22: 86581 states, bestScore=10
   minute 23: 17039 states, bestScore=12
   minute 24: 276 states, bestScore=12
   ```
   ```
   minute 18: 47281 states, bestScore=0
   minute 19: 82378 states, bestScore=6
   minute 20: 140294 states, bestScore=6
   minute 21: 239863 states, bestScore=10
   minute 22: 54829 states, bestScore=10
   minute 23: 9831 states, bestScore=12
   minute 24: 92 states, bestScore=12
   ```
3. If we could construct a type of robot in the last minute but we didn't do it, don't construct that type of robot in current minute either. (_5.7s_)

   ```
   minute 16: 626 states, bestScore=0
   minute 17: 1047 states, bestScore=0
   minute 18: 1756 states, bestScore=0
   minute 19: 2823 states, bestScore=6
   minute 20: 5021 states, bestScore=6
   minute 21: 9677 states, bestScore=10
   minute 22: 7615 states, bestScore=10
   minute 23: 2330 states, bestScore=12
   minute 24: 55 states, bestScore=12
   ```


In [2]:
import re
import numpy as np


def hashedState(state):
    resources, robots, _ = state
    return str(resources) + str(robots)


def stateScore(state, remainingMinutes):
    """
    returns 
        1. geodes can be collected with already built geode robots
        2. the upper bound of geodes that will be collected
    """
    # heuristic: the upper bound of geodes will be collected if we can construct a geode robot in each remaining minute
    #      18, 19, 20, 21, 22, 23, 24 # m
    #       6,  5,  4,  3,  2,  1,  0 # geodes collected by the robot constructed in minute m
    #      21, 15, 10,  6,  3,  1,  0 # total geodes can be collected
    # heuristic = (maxMinute - m) * (maxMinute - m + 1) // 2
    heuristic = remainingMinutes * (remainingMinutes + 1) // 2
    resources, robots, _ = state
    geodes = resources[-1] + robots[-1] * remainingMinutes
    return geodes, geodes + heuristic


def collectMaxGeodes(robotCost, maxMinute=24):
    robotsInc = np.identity(4, int)
    # BFS state = (resourceCount, robotCount, robotBuiltInLastMinute) in allowed time
    currentLevel = [(np.array([0, 0, 0, 0]), np.array([1, 0, 0, 0]), -1)]
    # max upper bound of states seen
    bestScore = 0
    # max cost of each type per minute
    maxCosts = np.max(robotCost, axis=0)
    for m in range(1, maxMinute + 1):
        nextLevel = []
        seen = set()
        # print(f"minute {m}: {len(currentLevel)} states, bestScore={bestScore}")
        for resources, robots, robotIdx in currentLevel:
            for i, cost in reversed(list(enumerate((robotCost)))):
                if np.all(cost <= resources):
                    # we can construct a robot of type i in the last minute, but we didn't do it
                    if robotIdx == -1 and all(cost <= resources - robots):
                        continue
                    if i == 3 or (
                        robots[i] < maxCosts[i]
                        and (
                            resources[i] + robots[i] * (maxMinute - m)
                            <= ((maxMinute - m) * maxCosts[i])
                        )
                    ):
                        newState = (resources - cost + robots, robots + robotsInc[i], i)
                        key = hashedState(newState)
                        # upper bound of geodes collected with the new state
                        score, estimated = stateScore(newState, maxMinute - m)
                        if estimated >= bestScore and key not in seen:
                            bestScore = max(score, bestScore)
                            nextLevel.append(newState)
                            seen.add(key)
            # case when no robot is constructed during this minute
            newState = (resources + robots, robots, -1)
            key = hashedState(newState)
            score, estimated = stateScore(newState, maxMinute - m)
            if estimated >= bestScore and key not in seen:
                nextLevel.append(newState)
                seen.add(key)
                bestScore = max(score, bestScore)

        currentLevel = nextLevel
    return max(resourceCount[-1] for resourceCount, *_ in currentLevel)


qualityLevels = {}
blueprints = []
with open("19.input", "r") as f:
    for blueprint in f:
        id, *costs = map(int, re.findall(r"\d+", blueprint))
        robotCosts = [
            # np.array([ore, clay, obsidian, geode])
            np.array([costs[0], 0, 0, 0]),  # ore robot
            np.array([costs[1], 0, 0, 0]),  # clay robot
            np.array([costs[2], costs[3], 0, 0]),  # obsidian robot
            np.array([costs[4], 0, costs[5], 0]),  # geode robot
        ]
        blueprints.append((id, robotCosts))

for id, robotCosts in blueprints:
    maxGeodes = collectMaxGeodes(robotCosts)
    print(f"{id}: {maxGeodes}")
    qualityLevels[id] = maxGeodes
print(sum(i * v for i, v in qualityLevels.items()))


1: 3
2: 1
3: 0
4: 0
5: 9
6: 0
7: 6
8: 2
9: 6
10: 4
11: 2
12: 0
13: 0
14: 1
15: 0
16: 3
17: 0
18: 3
19: 0
20: 2
21: 3
22: 3
23: 8
24: 1
25: 16
26: 0
27: 5
28: 7
29: 3
30: 1
1565


In [4]:
qualityLevels = {}
for id, robotCosts in blueprints[:3]:
    maxGeodes = collectMaxGeodes(robotCosts, 32)
    print(f"{id}: {maxGeodes}")
    qualityLevels[id] = maxGeodes
print(np.prod(list(qualityLevels.values())))


1: 29
2: 23
3: 16
10672
